## Logistic regression

In the first part of this exercise, we'll build a logistic regression model to predict whether a patient will going to get a cardiac arrest.  Suppose that you are a cardiologist and you want to determine each patient's chance of getting a heart attack based on the body measurments. You have historical data from previous patients that you can use as a training set for logistic regression.  To accomplish this, we're going to build a classification model that estimates the probability of admission based on the exam scores.

Let's start by examining the data.

In [ ]:
#!/usr/bin/env python3

##########################################################
# Copyright (c) Jesper Vang <jesper_vang@me.com>         #
# Created on 3 Aug 2021                                 #
# Version:	0.0.1                                        #
# What:  						                         #
##########################################################

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

print("Pandas Version: {}".format(pd.__version__))
print("Numpy Version: {}".format(np.__version__))
print("Matplotlib Version: {}".format(matplotlib.__version__))
print("Scikit-learn Version: {}".format(sklearn.__version__))
print("Seaborn Version: {}".format(sns.__version__))
np.set_printoptions(suppress=True, linewidth=130)


%matplotlib inline

plt.rcParams["figure.figsize"] = (10, 8)

print(f'the present working directory is: {os.getcwd()}')
DATA_PATH = os.path.join("data")
PROJECT_ROOT_DIR = "."

def load_model_data(data):
    pwd = os.getcwd() 
    filepath = os.path.join(pwd, DATA_PATH, data) 
    return pd.read_csv(filepath)     
    
data = load_model_data("heart.csv");




## Explore the data a bit.
### List:

1.   First few rows
2.   Basic statistic
3.   .info()
4.   Column names


In [ ]:
features = np.shape(data)[0]
samples = np.shape(data)[1]
print(f"Shape of Dataset: {samples} x {features}\n\t* Number of samples:\t{samples}\n\t* Number of features:\t{features}")

In [ ]:
# 1. First few rows
data.head()

In [ ]:
# 2. Basic statistics
data.describe()

In [ ]:
# 3. info()
data.info()

In [ ]:
# 4. Column names
columns = data.columns.to_list()

Questions:

1. What is the dependent variable (column name)?

It is the 'target'.

2. What are the independent variables?

  The rest of the variables in the data set, 

  ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

3. Should we normalize the data?

No, we don't need to normalize it.

4. What are the column data-types?

All columns are integers except for the "oldpeak", which is float. 

# Exploratory Data Analysis (EDA)

Create some simple plots to check out the data!

1.   Plot the pairwise scatter-plot between each column
2.   Plot the distribution of the values of the dependent variable
3.   Plot the pairwise correlation heatmap of each column.

Answer questions:

1.  What are the assumptions of the linear regression model?
2.  Can we accept the basic assumptions of the linear regression model?
3.  Judging by the scatter-plots, do you see any patterns in the data?
4.  Judging by the correlation heat-map, is there correlation between the dependent variable and the independent variables?
5.  Are there correlations among independent variables?

In [ ]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        dataframe[feature].hist(bins=20,ax=ax,facecolor='midnightblue')
        ax.set_title(feature+" Distribution",color='DarkRed')
        
    fig.tight_layout()  
    plt.show()
#draw_histograms(data,data.columns,6,3)

In [ ]:
# sns.pairplot(data=data)

First we need to create a sigmoid function.

Task:
1.  Implement the function

Make sure the function is correctly implemented.

Task:
2.  Plot the function.

In [ ]:
def sigmoid(z):
# Activation function used to map any real value between 0 and 1
    return 1/(1+ np.exp(-z))
# test function    
z = np.linspace(-10,10,num = 1000)
fig = plt.figure(figsize = (5,2))
sns.set(style = 'whitegrid')
sns.lineplot(x = z, y = sigmoid(z))

Task:
1. Separate the data to `X` and `y` arrays.
2. Separate the training set and evaluation set.
3. Check the shape of our arrays to make sure everything looks good.

In [ ]:
y = data["target"] # The dependent variable is selected as y
X = data.drop(["target"], axis=1)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f" Shape: x_train {x_train.shape} x_test: {x_test.shape}")
print(f" Shape: y_train {y_train.shape} y_test: {y_test.shape}")

In [ ]:
theta = [0.5]*len(X.columns)
theta

In [ ]:
def hypothesis(theta, X):
    z = np.dot(theta, X.T)
    return 1/(1+np.exp(-(z))) - 0.0000001

Task:
1. Write the cost function to evaluate a solution.

In [ ]:
def cost(theta, X, y):
    y1 = hypothesis(X, theta)
    return -(1/len(X)) * np.sum(y*np.log(y1) + (1-y)*np.log(1-y1))

Task:
1. Compute the cost for our initial solution (eyeball the initial value, e.g. zero).
2. Implement a function to compute the gradient (parameter updates) given our training data, labels, and model parameters.

In [ ]:
# number of elements
nr_features = x_train.shape[1] # = 13

nr = X.shape[0] # = 1000
m = len(X) # = 303

# initializing the theta values like 0.0. It can be initialized as for any other value
init_theta = np.zeros(x_train.shape[1], dtype=float)  # initializing theta as zeroes

# Learning Rate
alpha = 0.00005
# Number of iterations
epochs = 200000

def initial_solutions(init_theta, x_train, y_train):
    cost_func_value = cost(init_theta, x_train, y_train)  # calculating the initial cost
    print(f'The initial cost is: {cost_func_value:.2f}')

initial_solutions(init_theta, x_train, y_train)
#cost(init_theta, x_train, y_train)
#len(hypothesis(x_train,init_theta))
#len(sigmoid(np.dot(x_train, init_theta)))

In [ ]:
def gradient_descent(theta, X, y, alpha, epochs):
    m = len(X)
    J = [cost(theta, X, y)]
    for i in range(0, epochs):
        if i % 25 == 0: 
        print('i = {}'.format(i))
        h = hypothesis(X, theta)
        for i in range(0, len(X.columns)):
            theta[i] -= (alpha / m) * np.sum((h - y) * X.iloc[:, i])
        J.append(cost(theta, X, y))
    return J, theta


# gradient_descent(init_theta, x_train, y_train, alpha, epochs)


In [77]:
L, theta = gradient_descent(init_theta, x_train, y_train, alpha, epochs)


In [ ]:
def predict(theta, X, y, alpha, epochss):
    J, th = gradient_descent(theta, X, y, alpha, epochs) 
    h = hypothesis(X, theta)
    for i in range(len(h)):
        h[i]=1 if h[i]>=0.5 else 0
    y = list(y)
    acc = np.sum([y[i] == h[i] for i in range(len(y))])/len(y)
    return J, acc

In [ ]:
L, theta = gradient_descent(init_theta, x_train, y_train, alpha, epochs)


In [ ]:
predict(init_theta, x_train, y_train, alpha, epochs)

Task:
1. Calculate the cost for the optimized parameters

In [ ]:
theta = [0.5]*len(X.columns)
J, acc = predict(init_theta, x_train, y_train, alpha, epochs)

In [ ]:
acc

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize = (12, 8))
plt.scatter(range(0, len(J)), J)
plt.xlabel("Epochs")
plt.ylabel("Cost")
plt.show()

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)
print(f"Accuracy of logistic regression classifier on test set: {logreg.score(x_test, y_test):.2f}")

In [ ]:
print(logreg.coef_)
cost(logreg.coef_[0], x_train, y_train)

Task:
1. Write a function that will output predictions for a dataset X using our learned parameters.
2. Use this function to score and print the training accuracy of our classifier.

In [ ]:
def predict(x, theta):
  return sigmoid(np.dot(x, theta))

In [ ]:
# Hint: Accuracy is calculated  like: correctly classified samples / all samples
pred = np.round(predict(x_test, updated_weights))
accuracy = (y_test == pred).sum() / float(len(y_test))
print(f"The accuracy is: {accuracy}")

# Hint: Accuracy is calculated  like: correctly classified samples / all samples

pred = np.round(predict(x_test, updated_weights))

# I find the accuracy
accuracy = (y_test == pred).sum() / float(len(y_test))

print("The accuracy is: ", accuracy)

In [ ]:
plt.scatter(X_test.index,X_test.values,c=y_predict_test)
plt.show()